In [ ]:
!pip install -q ultralytics

In [ ]:
import cv2, numpy as np, random
from ultralytics import YOLO
from pathlib import Path

In [ ]:
YOLO_MODEL_PATH = "best.pt"
IMAGE_PATHS = [f"/content/img{i}.jpg" for i in range(1, 8)]
FG_SCALE = 0.173

In [ ]:
BASE_OUT = Path("/content")
OUT_DIR = BASE_OUT / "output"
(OUT_DIR / "masks").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "segmentations").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "seeds").mkdir(parents=True, exist_ok=True)

In [ ]:
model = YOLO(YOLO_MODEL_PATH)
print("Model loaded")

In [ ]:
def grabcut_constrained(img_path):
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        print(f"Can't read {img_path}")
        return

    h, w = img_bgr.shape[:2]
    results = model(img_bgr, verbose=False)
    bboxes = [b.xyxy.cpu().numpy().astype(int)[0] for b in results[0].boxes]
    if not bboxes:
        print(f"No hands detected in {img_path}")
        return

    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l_neutral = np.full_like(l, 128)
    img_for_grabcut = cv2.merge([l_neutral, a, b])

    mask = np.full((h, w), cv2.GC_PR_BGD, np.uint8)
    all_mask = np.zeros((h, w), np.uint8)
    vis = img_bgr.copy()

    for x1, y1, x2, y2 in bboxes:
        cv2.rectangle(vis, (x1, y1), (x2, y2), (255, 0, 0), 2)

        mask[y1:y2, x1:x2] = cv2.GC_PR_FGD
        all_mask[y1:y2, x1:x2] = 1

        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        sw, sh = int((x2 - x1) * FG_SCALE), int((y2 - y1) * FG_SCALE)
        sx1, sy1 = max(0, cx - sw//2), max(0, cy - sh//2)
        sx2, sy2 = min(w, cx + sw//2), min(h, cy + sh//2)
        mask[sy1:sy2, sx1:sx2] = cv2.GC_FGD
        cv2.rectangle(vis, (sx1, sy1), (sx2, sy2), (0, 255, 0), 2)

    bg_y, bg_x = np.where(all_mask == 0)
    n = min(1000, len(bg_y))
    idx = np.random.choice(len(bg_y), n, replace=False)
    mask[bg_y[idx], bg_x[idx]] = cv2.GC_BGD
    for x, y in zip(bg_x[idx], bg_y[idx]):
        cv2.circle(vis, (x, y), 1, (0, 0, 255), -1)

    mask_outside = np.ones_like(mask)
    for x1, y1, x2, y2 in bboxes:
        mask_outside[y1:y2, x1:x2] = 0
    mask[mask_outside == 1] = cv2.GC_BGD

    bgd, fgd = np.zeros((1,65), np.float64), np.zeros((1,65), np.float64)
    for _ in range(10):
        cv2.grabCut(img_for_grabcut, mask, None, bgd, fgd, 5, cv2.GC_INIT_WITH_MASK)

    binary = np.where((mask==cv2.GC_BGD)|(mask==cv2.GC_PR_BGD), 0, 1).astype(np.uint8)

    name = Path(img_path).name
    cv2.imwrite(str(OUT_DIR/"masks"/name), binary * 255)
    cv2.imwrite(str(OUT_DIR/"segmentations"/name),
                cv2.bitwise_and(img_bgr, img_bgr, mask=binary))
    cv2.imwrite(str(OUT_DIR/"seeds"/name), vis)

In [ ]:
for img_path in IMAGE_PATHS:
    print(f"🖼 Processing {img_path}...")
    grabcut_constrained(img_path)

print("\n Done! Check:")
print(" - /content/output_3/masks/")
print(" - /content/output_3/segmentations/")
print(" - /content/output_3/seeds/ (blue=YOLO box, green=FG, red=BG)")